In [36]:
import fitz  # PyMuPDF
import re
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

def extract_keywords(sentence):
    """문장에서 주요 키워드 추출"""
    words = re.findall(r'\b\w+\b', sentence)  # 단어 추출
    stop_words = {"및", "또는", "등", "관련", "하기", "있는", "이", "그", "를", "로", "과", "의"}  # 불용어 제거
    keywords = [word for word in words if word not in stop_words]
    return keywords[:5]  # 상위 5개 키워드 반환

def clean_and_split_text(text):
    """텍스트를 정리하고 문장 단위로 분할"""
    # 불필요한 패턴 제거
    text = re.sub(r"처음으로\(목차로 이동\)", "", text, flags=re.IGNORECASE)
    text = re.sub(r"처음으로", "", text, flags=re.IGNORECASE)
    text = re.sub(r"(목차로 이동)", "", text, flags=re.IGNORECASE)
    text = re.sub(r"(목차로이동)", "", text, flags=re.IGNORECASE)
    # 불필요한 기호 제거
    text = re.sub(r"[○•·▶◆\-\*]", "", text)  # 필요한 기호를 추가로 포함 가능

    text = re.sub(r"\n+", "\n", text)  # 여러 줄바꿈을 한 줄로 정리
    text = re.sub(r"\s+", " ", text)  # 다중 공백 제거
    sentences = text.split(". ")  # 문장 단위로 분할
    return sentences

def extract_first_phrase(text):
    """텍스트에서 첫 번째 문장 추출 (숫자 제외)"""
    lines = text.splitlines()
    for line in lines:
        # 첫 번째 유효한 문장 찾기
        if line.strip():
            match = re.match(r"^[^\d]+", line.strip())  # 숫자로 시작하지 않는 텍스트
            if match:
                return match.group(0).strip()
    return "N/A"

def process_pdf_to_new_docs(pdf_path):
    """
    PDF 파일을 처리하여 Document 객체 리스트를 생성합니다.

    Args:
        pdf_path (str): PDF 파일 경로.

    Returns:
        list: Document 객체 리스트.
    """
    doc = fitz.open(pdf_path)
    new_docs = []

    for page_number in range(len(doc)):
        if page_number < 2:
            continue
        page = doc[page_number]
        page_text = page.get_text()

        # 첫 번째 단어 추출
        first_word = extract_first_phrase(page_text)

        # 페이지 텍스트 정리 및 청크 분리
        cleaned_text = clean_and_split_text(page_text)
        splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
        chunks = splitter.split_text(" ".join(cleaned_text))

        # 청크별 Document 객체 생성 및 메타데이터 추가
        for idx, chunk in enumerate(chunks):
            keywords = extract_keywords(chunk)
            document = Document(
                metadata={
                    "first_word": first_word,
                    "year": 2025,  # 연도는 2025로 고정 (추가적으로 동적으로 추출 가능)
                    # "keywords": keywords,
                    # "chunk_id": idx,
                },
                page_content=chunk.strip()
            )
            new_docs.append(document)

    return new_docs

# PDF 파일 경로
pdf_path = "data/tax_etc/귀속_키워드연말정산.pdf"

# 문서 처리 및 메타데이터 추가
new_docs = process_pdf_to_new_docs(pdf_path)

# 결과 확인
for doc in new_docs:
    print(doc.metadata)
    print(doc.page_content)

print()


{'first_word': '가산세', 'year': 2025}
1 가산세 () 근로소득지급명세서관련가산세 미제출: 제출하지아니한분의지급금액의1% 불분명: 불분명또는사실과다른금액의1% 지연제출: 3개월이내에제출하는경우지급금액의0.5% 한도: 1억원(중소기업기본법§2①에따른중소기업은5천만원) 단, 고의적 위반의경우한도없음 원천징수관련가산세 (1) 원천징수등납부지연가산세 국세를 징수하여 납부할 의무를 지는 자가 징수하여야 할 세액을 법정납부기한까지 납부하지 아니하거나 과소납부한 경우에는 납부하지 아니한 세액 또는 과소납부분 세액의 100분의 50(①과 ② 중 법정납부기한의 다음 날부터 납세고지일까지의 기간에 해당하는 금액을 합한 금액은 100분의 10)에 상당하는 금액을 한도로 하여 다음 금액을 합한 금액을 가산세로 부과함 ① 납부하지 아니한 세액 또는 과소납부분 세액의 100분의 3에 상당하는 금액 ② 납부하지 아니한 세액 또는 과소납부분 세액 × 법정납부기한의 다음 날부터 납부일까지의 기간 (납세고지일부터 납세고지서에 따른 납부기한까지의 기간은 제외한다) × 2.2/10,000 미납세액× 3% ＋(과소⋅무납부세액× 2.2/10,000 × 경과일수) ≦50% (단, 법정납부기한의 다음날부터 고지일까지의 기간에 해당하는 금액≦10%) (2) 원천징수등납부지연가산세적용제외 우리나라에주둔하는미군 공적연금(국민연금, 공무원연금등) 관련법에따라연금일시금을지급하는자 국가지방자치단체또는지방자치단체조합 단, 국가등으로부터근로소득을받는사람이근로소득자소득공제신고서를 사실과다르게기재하여부당하게소득공제를받아국가등이원천징수하여야 할세액을정해진기간에납부하지아니하거나미달하게납부한경우에는 국가등은징수하여야할세액에원천징수등납부지연가산세에상당하는 금액을더하여그근로소득자로부터징수하여납부하여야함 면제특례(조특법§ 126의2 ⑤) 원천징수의무자가근로자소득공제신고서및신용카드소득공제신청서에 기재된신용카드사용금액에신용카드등사용금액에서제외되는금액이 포함되어있음을연말정산시까지확인할수없는경우그사유로원천징수 하